In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI()

In [4]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):
    prompt = f"generate a joke on the following topic: {state["topic"]}"
    response = llm.invoke(prompt)
    return {'joke': response.content}

def generate_explanation(state: JokeState):
    prompt = f"write an explanation for the joke: {state['joke']}"
    response = llm.invoke(prompt)
    return {'explanation': response.content}

In [6]:
checkpointer = InMemorySaver()
# Create graph
graph = StateGraph(state_schema=JokeState)

# Create nodes
graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explanation", generate_explanation)

# Create edges
graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke","generate_explanation")
graph.add_edge("generate_explanation", END)

# compile graph
workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
final_state = workflow.invoke({"topic": "american football"}, config=config1)
print(final_state)

{'topic': 'american football', 'joke': 'Why did the football coach go to the bank?\n\nTo get his quarterback!', 'explanation': 'This joke plays on the double meaning of the word "quarterback." In football, the quarterback is a position on the field responsible for leading the team\'s offense. However, in financial terms, a quarterback can also refer to a backer of a bank loan or investment. Therefore, the joke suggests that the football coach went to the bank to get his quarterback for the game, but in reality, he went to get a financial backer or loan.'}


In [8]:
workflow.get_state(config=config1)

StateSnapshot(values={'topic': 'american football', 'joke': 'Why did the football coach go to the bank?\n\nTo get his quarterback!', 'explanation': 'This joke plays on the double meaning of the word "quarterback." In football, the quarterback is a position on the field responsible for leading the team\'s offense. However, in financial terms, a quarterback can also refer to a backer of a bank loan or investment. Therefore, the joke suggests that the football coach went to the bank to get his quarterback for the game, but in reality, he went to get a financial backer or loan.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f086a8e-cea5-6e1f-8002-a489ee54f550'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-31T20:27:33.589535+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f086a8e-bb43-6751-8001-24afb9f7263d'}}, tasks=(), interrupts=())

In [9]:
list(workflow.get_state_history(config=config1))

[StateSnapshot(values={'topic': 'american football', 'joke': 'Why did the football coach go to the bank?\n\nTo get his quarterback!', 'explanation': 'This joke plays on the double meaning of the word "quarterback." In football, the quarterback is a position on the field responsible for leading the team\'s offense. However, in financial terms, a quarterback can also refer to a backer of a bank loan or investment. Therefore, the joke suggests that the football coach went to the bank to get his quarterback for the game, but in reality, he went to get a financial backer or loan.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f086a8e-cea5-6e1f-8002-a489ee54f550'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-31T20:27:33.589535+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f086a8e-bb43-6751-8001-24afb9f7263d'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'t